## <b>Machine Learning</b> <i>Titanic</i><br><br>

<b>PassengerId:</b> um identificador único para cada passageiro.<br>
<b>Survived:</b> a resposta que queremos prever. 1 = sobreviveu, 2 = não sobreviveu.<br>
<b>Pclass:</b> identifica a classe do bilhete do passageiro. 1 = primeira classe, 2 = segunda classe, 3 = terceira classe. Esta variável tem grande relação com o status socioeconômico do passageiro. Primeira classe = alto, segunda classe = médio, terceira classe = baixo.<br>
<b>Name:</b> nome do passageiro.<br>
<b>Sex:</b> sexo do passageiro.<br>
<b>Age:</b> idade em anos. Se a idade do passageiro for menor que 1, então a idade terá um valor fracional (ex.: 0.75). Se a idade do passageiro for estimada, então ela terá o formato XX.5 (ex.: 35.5 para uma idade estimada de 35 anos).<br>
<b>SibSp:</b> quantidade de irmãos e cônjuges do passageiro que também estão abordo.<br>
<b>Parch:</b> quantidade de pais e filhos do passageiro que também estão abordo.<br>
<b>Ticket:</b> número do bilhete do passageiro.<br>
<b>Fare:</b> valor da tarifa paga pelo passageiro.<br>
<b>Cabin:</b> número da cabine do passageiro.<br>
<b>Embarked:</b> porto onde o passageiro embarcou. C = Cherbourg, Q = Queenstown, S = Southampton.<br>

In [11]:
import pandas as pd
from typing import Tuple
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

class DataPrep:
  def __init__(self, data: pd.DataFrame) -> None:
    """Inicializa a classe DataPrep com a base de dados do Titanic."""
    self.data= data

  def remover_variaveis(self) -> None:
    """Remove as variáveis que não serão utilizadas pelo modelo."""
    colunas_para_remover= [
        "PassengerId",
        "Name",
        "Ticket",
        "Cabin", # Variável com muitos dados faltantes
        "Embarked", # Substituídas por variáveis dummies
        "SibSp", "Parch" # Foram combinadas em uma nova variável
    ]
    self.data.drop(columns= colunas_para_remover, inplace= True)

  def tratar_nulos(self) -> None:
    """Faz o tratamento das variáveis nulas, imputando o valor adequado."""
    self.data["Age"] = self.data.groupby(["Pclass", "Sex"], group_keys= False)["Age"].apply(lambda x: x.fillna(x.median()))
    self.data["Embarked"]= self.data["Embarked"].fillna("S")

  def tratar_variaveis_categoricas(self) -> None:
    """Faz o tratamento das variáveis categóricas."""
    sexo= {"male": 0, "female": 1} # Label Encoding
    self.data["Sex"]= self.data["Sex"].map(sexo)

    embarked_dummies= pd.get_dummies(self.data["Embarked"])
    self.data= pd.concat([self.data, embarked_dummies], axis= 1)

  def dimensionar_dados(self) -> None:
    variaveis= self.data.drop(columns= "Survived")
    var_cols= variaveis.columns
    resposta= self.data["Survived"]

    scaler= MinMaxScaler()
    variaveis= scaler.fit_transform(variaveis)
    variaveis= pd.DataFrame(variaveis, columns= var_cols)
    self.data= pd.concat([variaveis, resposta], axis= 1)

  def criar_variaveis(self) -> None:
    self.data["FamilySize"]= self.data["SibSp"] + self.data["Parch"] + 1

  def separa_treino_teste(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Separa a base de dados entre conjunto de treinamento e teste."""
    treino, teste= train_test_split(self.data, test_size= 0.3, random_state= 2021)
    return treino, teste

  def preparar_dados(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Executa todas as etapas de transformação de dados."""
    self.tratar_nulos()
    self.tratar_variaveis_categoricas()
    self.criar_variaveis()
    self.remover_variaveis()
    self.dimensionar_dados()
    treino, teste= self.separa_treino_teste()
    return treino, teste

df = pd.read_csv('/content/drive/MyDrive/datasets/titanic_train.csv')

dp= DataPrep(df)

df_treino, df_teste= dp.preparar_dados()

df_treino.head()

,Pclass,Sex,Age,Fare,C,Q,S,FamilySize,Survived
725,1.0,0.0,0.246042,0.016908,0.0,0.0,1.0,0.0,0
52,0.0,1.0,0.610455,0.149765,1.0,0.0,0.0,0.1,1
883,0.5,0.0,0.346569,0.020495,0.0,0.0,1.0,0.0,0
344,0.5,0.0,0.447097,0.025374,0.0,0.0,1.0,0.0,0
450,0.5,0.0,0.447097,0.054164,0.0,0.0,1.0,0.3,0


AttributeError: ignored

In [ ]:
df.isna().sum() / df.shape[0] * 100 # Porcentagem de dados faltantes em cada variável

In [ ]:
df["Age"].median()

In [ ]:
df_idade= df.groupby(["Pclass", "Sex"], as_index= False, )["Age"].median()
df_idade

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["axes.spines.top"]= False
plt.rcParams["axes.spines.right"]= False

fig, ax= plt.subplots(figsize=(8, 5))
sns.barplot(x= "Pclass", y= "Age", hue= "Sex", data= df_idade)
ax.set_title("Idade por Classe e Sexo", fontsize= 16)
plt.show()

In [ ]:
# Uma estratégia viável é imputar variáveis categóricas com o valor mais frequente
df["Embarked"].value_counts()

In [ ]:
df["Survived"].value_counts(normalize= True)

In [ ]:
df_plot= df.groupby("Sex", as_index= False)["Survived"].mean()
fig, ax= plt.subplots(figsize=(8, 5))
sns.barplot(x= "Sex", y= "Survived", data= df_plot)
ax.set_title("Taxa de sobrevivência por sexo", fontsize= 16)
ax.set_xticklabels(["Female", "Male"])
plt.show()

In [ ]:
familia= df['SibSp'] + df['Parch'] + 1
fig, ax= plt.subplots()
sns.barplot(x= familia, y= df['Survived'])
ax.set_title("Taxa de sobrevivência por tamanho da familia")
plt.show()